In [1]:
import pandas as pd
import numpy as np
from pdfminer.high_level import extract_text
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBox, LTTextLine, LTChar
from IPython.display import display
import fitz  # PyMuPDF

In [3]:
import pandas as pd
import numpy as np
from pdfminer.high_level import extract_text
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBox, LTTextLine, LTChar
from IPython.display import display
import fitz  # PyMuPDF

def extract_metadata(pdf_file):
    try:
        document = fitz.open(pdf_file)
        metadata = document.metadata
        return metadata
    except Exception as e:
        print(f"Error extracting metadata from {pdf_file}: {e}")
        return {}

def extract_font_information(pdf_file):
    font_data = []
    try:
        with open(pdf_file, 'rb') as file:
            rsrcmgr = PDFResourceManager()
            laparams = LAParams()
            device = PDFPageAggregator(rsrcmgr, laparams=laparams)
            interpreter = PDFPageInterpreter(rsrcmgr, device)
            for page_number, page in enumerate(PDFPage.get_pages(file), start=1):
                interpreter.process_page(page)
                layout = device.get_result()
                for element in layout:
                    if isinstance(element, (LTTextBox, LTTextLine)):
                        for text_line in element:
                            line_text = text_line.get_text().strip()
                            for char in text_line:
                                if isinstance(char, LTChar):
                                    # Handle color extraction
                                    color = getattr(char.graphicstate, 'ncolor', 'Unknown')
                                    if isinstance(color, tuple):
                                        color = sum(color) / len(color)  # Calculate the average of the tuple values
                                    elif isinstance(color, np.ndarray):
                                        color = np.mean(color) 
                                    elif isinstance(color, list):
                                        color = sum(color) / len(color)

                                    
                                    font_data.append({
                                        "Text": str(line_text),
                                        "Font Style": str(char.fontname),
                                        "Font Size": int(char.size),
                                        "Color": float(color)
                                    })
    except Exception as e:
        print(f"Error processing {pdf_file}: {e}")
    return font_data

def extract_font_information_with_metadata(pdf_file):
    # Extract metadata
    metadata = extract_metadata(pdf_file)
    producer = str(metadata.get('producer', 'Unknown'))

    # Extract font information
    font_data = extract_font_information(pdf_file)
    
    # Add metadata 'producer' field to each record
    for entry in font_data:
        entry['Producer'] = producer
    
    return font_data

# Extract font information with metadata from both PDFs
pdf_file1 = "./certificates_dataset/Cybersecurity_Certf.pdf"
font_info1 = extract_font_information_with_metadata(pdf_file1)
font_info_df1 = pd.DataFrame(font_info1)

pdf_file2 = "./certificates_dataset/Redhat_Certf.pdf"
font_info2 = extract_font_information_with_metadata(pdf_file2)
font_info_df2 = pd.DataFrame(font_info2)

# Remove duplicate entries
font_info_df1_unique = font_info_df1.drop_duplicates()
font_info_df2_unique = font_info_df2.drop_duplicates()

# Display the unique DataFrames
print("Unique Font Information for random.pdf:")
display(font_info_df1_unique)

print("Unique Font Information for Soham  Dalvi 623593.pdf:")
display(font_info_df2_unique)


Unique Font Information for random.pdf:


,Text,Font Style,Font Size,Color,Producer
0,Sumit Ganesh Mesta,Helvetica,20,0.207840,iText® 5.5.12 ©2000-2017 iText Group NV (AGPL-...
18,A. P. Shah Institute of Technology,Helvetica,15,0.150327,iText® 5.5.12 ©2000-2017 iText Group NV (AGPL-...
52,Certificate ID :105173542b77279971c9fa0a05fad78a,Helvetica,10,0.000000,iText® 5.5.12 ©2000-2017 iText Group NV (AGPL-...
100,Student ID :STU62d65aee5f4fe1658215150,Helvetica,10,0.000000,iText® 5.5.12 ©2000-2017 iText Group NV (AGPL-...


Unique Font Information for Soham  Dalvi 623593.pdf:


,Text,Font Style,Font Size,Color,Producer
0,Sumit Mesta,TYSYXI+OpenSans-Regular,16,0.2,iText 2.1.7 by 1T3XT
11,77,TYSYXI+OpenSans-Regular,16,0.2,iText 2.1.7 by 1T3XT


In [7]:
import pandas as pd
from pdfminer.high_level import extract_text
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBox, LTTextLine, LTChar
from IPython.display import display
import fitz, os

def extract_metadata(pdf_file):
    try:
        document = fitz.open(pdf_file)
        metadata = document.metadata
        return metadata
    except Exception as e:
        print(f"Error extracting metadata from {pdf_file}: {e}")
        return {}

def extract_font_information(pdf_file):
    font_data = []
    structure = {
        "number_of_pages": 0,
        "fonts": set(),
        "font_colors": [],
    }
    try:
        with open(pdf_file, 'rb') as file:
            rsrcmgr = PDFResourceManager()
            laparams = LAParams()
            device = PDFPageAggregator(rsrcmgr, laparams=laparams)
            interpreter = PDFPageInterpreter(rsrcmgr, device)
            for page_number, page in enumerate(PDFPage.get_pages(file), start=1):
                interpreter.process_page(page)
                layout = device.get_result()
                for element in layout:
                    if isinstance(element, (LTTextBox, LTTextLine)):
                        for text_line in element:
                            line_text = text_line.get_text().strip()
                            for char in text_line:
                                if isinstance(char, LTChar):
                                    # Handle color extraction
                                    color = getattr(char.graphicstate, 'ncolor', 'Unknown')
                                    if isinstance(color, tuple):
                                        color = sum(color) / len(color)  # Calculate the average of the tuple values
                                    elif isinstance(color, np.ndarray):
                                        color = np.mean(color) 
                                    elif isinstance(color, list):
                                        color = sum(color) / len(color)
                                    
                                    font_data.append({
                                        "Text": str(line_text),
                                        "Font Style": str(char.fontname),
                                        "Font Size": float(char.size),
                                        "Color": float(color),
                                        "Label" : int(1)
                                    })

                                    # Add font information to structure
                                    font_info = (char.fontname, char.size, color)
                                    structure["fonts"].add(char.fontname)
                                    structure["font_colors"].append(font_info)
                                    
    except Exception as e:
        print(f"Error processing {pdf_file}: {e}")
    return font_data

def extract_image_count(pdf_file):
    try:
        document = fitz.open(pdf_file)
        image_count = 0
        for page_num in range(len(document)):
            page = document[page_num]
            image_count += len(page.get_images(full=True))
        return image_count
    except Exception as e:
        print(f"Error counting images in {pdf_file}: {e}")
        return 0

def extract_font_information_with_metadata_and_images(pdf_file):
    # Extract metadata
    metadata = extract_metadata(pdf_file)
    producer = str(metadata.get('producer', 'Unknown'))

    # Extract font information
    font_data = extract_font_information(pdf_file)
    
    # Extract image count
    image_count = extract_image_count(pdf_file)
    
    # Add metadata 'producer' field and image count to each record
    for entry in font_data:
        entry['Producer'] = producer
        entry['Image Count'] = image_count
    
    return font_data

# Directory containing the PDF files
directory_path = "./certificates_dataset"

# Iterate through all PDF files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".pdf"):
        print("File : ", filename)
        pdf_file = os.path.join(directory_path, filename)
        pdf_ore = extract_font_information_with_metadata_and_images(pdf_file)
        pdf_mineral = pd.DataFrame(pdf_ore)
        pdf_mineral = pdf_mineral.drop_duplicates()
        pdf_mineral.to_csv("real_cert.csv", mode='a', header=not os.path.exists("real_cert.csv"), index=False)


File :  AIML_Cert (2) COMP.pdf
File :  AWS_Cloud_Virtual_Internship.pdf
File :  CelonisDefault20240423-31-gh2sdu.pdf
File :  CF_AWS_Academy_Graduate___AWS_Academy_Cloud_Foundations_Badge20230125-30-k2x1xb.pdf
File :  Cybersecurity_Certf.pdf
File :  Data_Analysis_Using_Python_Badge20240321-37-7tk5a0.pdf
File :  Fundamentals of Cloud Security _ Beacon.pdf
File :  Fundamentals of Network Security _ Beacon.pdf
File :  IBM DA0101EN Certificate _ Cognitive Class.pdf
File :  IIT SPOKEN_Python.pdf
Error processing ./certificates_dataset\IIT SPOKEN_Python.pdf: float() argument must be a string or a real number, not 'NoneType'
File :  IITSPOKEN_Linux.pdf
Error processing ./certificates_dataset\IITSPOKEN_Linux.pdf: float() argument must be a string or a real number, not 'NoneType'
File :  INC1.pdf
File :  INC2.pdf
File :  Introduction to Cybersecurity _ Beacon.pdf
File :  JNCAA-Certiciate of Attendance_comp.pdf
File :  JNCIA-Certiciate of Attendance_comp.pdf
File :  Juniper Networks Cloud Virtual

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix

# Load the dataset
data = pd.read_csv("real_cert.csv")

# Define features and target
X = data.drop(columns=['Label'])
y = data['Label']

# Categorical and numerical features
categorical_features = ['Font Style', 'Producer']
numerical_features = ['Font Size', 'Color', 'Image Count']

# Preprocessing pipelines
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define the model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Example: Predict on a new PDF's features
new_pdf_features = {
    'Text': "...", 
    'Font Style': "Random font", 
    'Font Size': 12, 
    'Color': 0.5, 
    'Producer': "Producer Name", 
    'Image Count': 2
}
new_pdf_df = pd.DataFrame([new_pdf_features])
predicted_label = model.predict(new_pdf_df)

if predicted_label == 0:
    print(f"The PDF is likely fake. Producer: {new_pdf_features['Producer']}")
else:
    print("The PDF is genuine.")


[[144]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       144

    accuracy                           1.00       144
   macro avg       1.00      1.00      1.00       144
weighted avg       1.00      1.00      1.00       144

The PDF is genuine.


c:\Users\abc\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
